# Notebook for predicting motor impairment severity using functional connectivity data

In this notebook, we train a simple MLP to predict motor impairment using either subnetworks or individual connections from fMRI data.

## Imports

In [5]:
from utils.data_loading import load_clinical_data, load_fmri_data, select_subnetwork
from utils.model import train_eval
import numpy as np
from scipy.stats import wilcoxon

## Directories

In [6]:
clinical_data_path = '../data/private_data/clinical_data.csv'
gait_scores_path = '../data/private_data/gait_scores.csv'
fc_matrices_dir = '../data/private_data/fc_matrices/'

## Predict motor impairment from FC (rs-fMRI) data

We use leave-one-out validation and average results across 20 runs.

In [8]:
clinical_data = load_clinical_data(clinical_data_path, gait_scores_path)
fmri_data = load_fmri_data(fc_matrices_dir)
networks = ['random', 'pallidal-sensorimotor', 'cerebellar-motor', 'cerebellar-prefrontal', 'top-cerebellar-prefrontal', 'top-whole-brain']
for network in networks:
    use_random_connections = False
    if network != 'random':
        network_data = select_subnetwork(fmri_data, clinical_data, network)
    else:
        network_data = fmri_data
        use_random_connections = True
        
    X = []
    y = []
    for _, row in clinical_data.iterrows():
        X.append(network_data[row['PID']])
        y.append(row['majority vote'] - 1)
    X_np = np.array(X)
    y_np = np.array(y)
    auc_scores, f1_scores, precision_scores, recall_scores = train_eval(X_np, y_np, use_random_connections)
    
    print(f'{network} stats - auc mean: {np.mean(auc_scores):.3f}, f1 mean: {np.mean(f1_scores):.3f}, precision mean: {np.mean(precision_scores):.3f}, recall mean: {np.mean(recall_scores):.3f}')
    if network == 'random':
        random_auc = np.mean(auc_scores)
    else:
        wilcoxon_output = wilcoxon(auc_scores, [random_auc] * len(auc_scores), alternative='greater')
        print(f'result sig: {wilcoxon_output.pvalue}')

random stats - auc mean: 0.455, f1 mean: 0.295, precision mean: 0.294, recall mean: 0.300
pallidal-sensorimotor stats - auc mean: 0.449, f1 mean: 0.328, precision mean: 0.346, recall mean: 0.346
result sig: 0.762547492980957
cerebellar-motor stats - auc mean: 0.521, f1 mean: 0.331, precision mean: 0.343, recall mean: 0.357
result sig: 6.67572021484375e-06
cerebellar-prefrontal stats - auc mean: 0.635, f1 mean: 0.386, precision mean: 0.394, recall mean: 0.391
result sig: 9.5367431640625e-07
top-cerebellar-prefrontal stats - auc mean: 0.724, f1 mean: 0.460, precision mean: 0.508, recall mean: 0.582
result sig: 9.5367431640625e-07


/deep/u/markendo/anaconda3/envs/myenv/lib/python3.7/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


top-whole-brain stats - auc mean: 0.738, f1 mean: 0.481, precision mean: 0.495, recall mean: 0.597
result sig: 9.5367431640625e-07
